- pdfをファイルから読み込み，outファイルに保存をできるようにする

In [6]:
import subprocess
import zenhan
import os
import sys
import re
import unicodedata
import argparse
import tqdm

In [7]:
def normalize(sentence):
    sentense = unicodedata.normalize('NFKC' , sentence)
    sentence = zenhan.h2z(sentense)
    sentence = sentence.replace(u' ', u'').replace(u'　', u'' )
    sentence = re.sub("①\n\n","1",sentence)
    #sentence = sentence.replace(u'\n', u'')
    return re.sub("\s*", u'', sentence)

In [8]:
def create_text(out,file):
    #個別
    #out='file/out/company2/rakuten.txt'
    #file = 'file/pdf/company2/rakuten.pdf'
    line_1 = ""
    text=""
    sentences_list=[]
    #print(file)
    #print(out)
    with open(out,'w') as out:
        mat_eol = re.compile("\n".encode('utf-8'))
        #print(fp)
        sentences = []
        sentence=[]
        p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        temp = u""
        #print(p.stdout.decode('utf-8'))
        for line in mat_eol.split(p.stdout):
            try:
                line = line.decode('utf-8')
            except:
                line = ""
            if line == "":
                if temp != "" and u"。" in temp:
                    sentences = sentences + re.split(u"。", temp)
                temp = u""
            else:
                temp = temp+line

        if temp != "" and u"。" in temp:
            sentences = sentences + re.split(u"。", temp)
            #print(sentences)
        #print(sentences)
        #sentences = [normalize(line + u"。") for line in sentences if len(line) <= 300 and len(line) >+ 3]
        for line in sentences :
            text=line
            if "①" in line_1 or "②" in line_1 or "③" in line_1 or "④" in line_1 or "⑤" in line_1:
                #print(line_1)
                text = "0" + line
            if len(text) <= 300 and len(text) >+ 3:
                sentences_list.append(normalize(text + u"。"))
                #print(text)
            line_1 = line
            #print(line_1)

        for sentence in sentences_list:
            #print(sentence)
            out.write(sentence+"\n")

In [ ]:
out='file/out/company2/rakuten.txt'
file = 'file/pdf/company2/rakuten.pdf'
create_text(out,file)

In [11]:
#一括
for i in range(0,1):
    for j in range(2):
        out='file/out/company{}/{}.txt'.format(i+1,j+1)
        file = 'file/pdf/company{}/{}.pdf'.format(i+1,j+1)
        #print(file)
        #print(out)
        with open(out,'w') as out:
            mat_eol = re.compile("\n".encode('utf-8'))
            #print(fp)
            sentences = []
            sentence=[]
            p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            temp = u""
            for line in mat_eol.split(p.stdout):
                try:
                    line = line.decode('utf-8')
                except:
                    line = ""
                if line == "":
                    if temp != "" and u"。" in temp:
                        sentences = sentences + re.split(u"。", temp)
                    temp = u""
                else:
                    temp = temp+line

            if temp != "" and u"。" in temp:
                sentences = sentences + re.split(u"。", temp)

            sentences = [normalize(line + u"。")
                        for line in sentences if len(line) <= 300 ]#and len(line) >+ 3]
            for sentence in sentences:
                out.write(sentence+"\n")

## １０文字以下の場合は丸をつけない
- こっちに処理を入れるとひょうとごっちゃになるから却下

In [3]:
#個別
out='file/out/company2/rakuten.txt'
file = 'file/pdf/company2/rakuten.pdf'
line_1 = ""
text=""
sentences_list=[]
#print(file)
#print(out)
with open(out,'w') as out:
    mat_eol = re.compile("\n".encode('utf-8'))
    #print(fp)
    sentences = []
    sentence=[]
    p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    temp = u""
    #print(p.stdout.decode('utf-8'))
    for line in mat_eol.split(p.stdout):
        try:
            line = line.decode('utf-8')
        except:
            line = ""
        if line == "":
            if temp != "" and u"。" in temp:
                sentences = sentences + re.split(u"。", temp)
            temp = u""
        else:
            temp = temp+line

    if temp != "" and u"。" in temp:
        sentences = sentences + re.split(u"。", temp)
        #print(sentences)
    #print(sentences)
    #sentences = [normalize(line + u"。") for line in sentences if len(line) <= 300 and len(line) >+ 3]
    for line in sentences :
        text=line
        if "①" in line_1 or "②" in line_1 or "③" in line_1 or "④" in line_1 or "⑤" in line_1:
            #print(line_1)
            text = "0" + line
        if len(text) <= 300 and len(text) >+ 3:
            sentences_list.append(normalize(text + u"。"))
            #print(text)
        line_1 = line
        #print(line_1)
        
    for sentence in sentences_list:
        #print(sentence)
        out.write(sentence+"\n")

# 目次を使用してのtext変更

In [9]:
import pandas as pd

In [10]:
#企業リストを取得
company_list = pd.read_csv("../data/new2/df_achieve/company_list.csv",index_col=0)
company_list=company_list.drop(14350)
company=13820
#company=16620
df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
df = pd.read_csv(df_path,index_col=0)
mokuzi_path = '../data/text_mokuzi_data_raw/'+str(company)+'/'+str(df["pdf_code"][0])+'.txt'
pdf_path = '../data/TDNet_full_data/API_data/'+str(company)+'_g/'+str(company)+'_'+str(df["pdf_code"][0])+'.pdf'

In [159]:
def text_normalize(text):
    text = text.replace('添付資料の目次','')
   #text = text.replace('…','')
    #text= text.replace('·','')
    text= text.replace('○','')
    text= text.replace('\n\n','')
    text= text.replace(' ','')
    text= text.replace('\x0c','')
    return(text)

In [174]:
with open(mokuzi_path,'r') as t:
    text=t.read()

In [175]:
text=(text_normalize(text))
Flag=False
mokuzi=[]
mokuzi_start=0
flag=False
for i,word in enumerate(text):
    if not flag:
        if word == "…" or word == '·' or word == '･':
            new_text=text[mokuzi_start:i].replace('\n','')
            mokuzi.append(new_text)
            flag=True
    
    elif flag and word=='\n':
        mokuzi_start=i+1
        flag=False

In [176]:
mokuzi

['１．経営成績等の概況',
 '（１）当期の経営成績の概況',
 '（２）当期の財政状態の概況',
 '（３）当期のキャッシュ・フローの概況',
 '（４）今後の見通し',
 '（５）継続企業の前提に関する重要事象等',
 '２．会計基準の選択に関する基本的な考え方',
 '３．連結財務諸表及び主な注記',
 '（１）連結貸借対照表',
 '（２）連結損益計算書及び連結包括利益計算書',
 '連結損益計算書',
 '連結包括利益計算書',
 '（３）連結株主資本等変動計算書',
 '（４）連結キャッシュ・フロー計算書',
 '（５）連結財務諸表に関する注記事項',
 '（継続企業の前提に関する注記）',
 '（連結財務諸表作成のための基本となる重要な事項）',
 '（追加情報）',
 '（連結貸借対照表関係）',
 '（連結損益計算書関係）',
 '（連結包括利益計算書関係）',
 '（連結株主資本等変動計算書関係）',
 '（連結キャッシュ・フロー計算書関係）',
 '（セグメント情報等）',
 '（１株当たり情報）',
 '（重要な後発事象）',
 '４．その他']

In [302]:
#個別
out='file/out/test.txt'
file = pdf_path
text=""
sentences_list=[]
#print(file)
#print(out)
with open(out,'w') as out:
    mat_eol = re.compile("\n".encode('utf-8'))
    #print(fp)
    sentences = []
    sentence=[]
    p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    temp = u""
    #print(p.stdout.decode('utf-8'))
    for line in mat_eol.split(p.stdout):
        try:
            line = line.decode('utf-8')
        except:
            line = ""
        if line == "":
            if temp != "" and u"。" in temp:
                sentences = sentences + re.split(u"。", temp)
            temp = u""
        else:
            temp = temp+line

    if temp != "" and u"。" in temp:
        sentences = sentences + re.split(u"。", temp)
     
    for line in sentences :
        text=line

        #if len(text) <= 300 and len(text) >+ 3:
        #if len(text) > 3:
        sentences_list.append(normalize(text + u"。"))
            #print(text)
        
    for sentence in sentences_list:
        #print(sentence)
        out.write(sentence+"\n")

In [178]:
out='file/out/test.txt'
with open(out,'r') as t:
    text = t.read()

In [179]:
mokuzi_list = mokuzi

In [180]:
mokuzi_list

['１．経営成績等の概況',
 '（１）当期の経営成績の概況',
 '（２）当期の財政状態の概況',
 '（３）当期のキャッシュ・フローの概況',
 '（４）今後の見通し',
 '（５）継続企業の前提に関する重要事象等',
 '２．会計基準の選択に関する基本的な考え方',
 '３．連結財務諸表及び主な注記',
 '（１）連結貸借対照表',
 '（２）連結損益計算書及び連結包括利益計算書',
 '連結損益計算書',
 '連結包括利益計算書',
 '（３）連結株主資本等変動計算書',
 '（４）連結キャッシュ・フロー計算書',
 '（５）連結財務諸表に関する注記事項',
 '（継続企業の前提に関する注記）',
 '（連結財務諸表作成のための基本となる重要な事項）',
 '（追加情報）',
 '（連結貸借対照表関係）',
 '（連結損益計算書関係）',
 '（連結包括利益計算書関係）',
 '（連結株主資本等変動計算書関係）',
 '（連結キャッシュ・フロー計算書関係）',
 '（セグメント情報等）',
 '（１株当たり情報）',
 '（重要な後発事象）',
 '４．その他']

In [300]:
text_list=[]
text_dic={"text":[]}
#kari_dic={}
kari_text=text
true_mokuzi=[]
false_mokuzi=[]
for i,mokuzi in enumerate(mokuzi_list):
    ind=kari_text.find(mokuzi)
    if ind!=-1:
        #目次があったものは変数に入れておく
        true_mokuzi.append(mokuzi)
        kari_dic={}
        if text_dic["text"]!=[]:
            kari_dic["title"]=true_mokuzi[-2]
            kari_dic["text"]=kari_text[len(true_mokuzi[-2]):ind]
            text_dic["text"].append(kari_dic)
            kari_text=kari_text[ind:]
            
            #mokuzi_listの最後に来た時の処理
            if mokuzi==mokuzi_list[-1]:
                kari_dic={}
                kari_dic["title"]=true_mokuzi[-1]
                kari_dic["text"]=kari_text[len(true_mokuzi[-1]):ind]
                text_dic["text"].append(kari_dic)
                break
        
        #false_mokuziに目次があるとき（textのない目次）
            if false_mokuzi!=[]:
                for non_mokuzi in false_mokuzi:
                    kari_dic={}
                    kari_dic["title"]=non_mokuzi
                    kari_dic["text"]=""
                    text_dic["text"].append(kari_dic)
                false_mokuzi=[]
        
        elif i==0:
            kari_dic["title"]="目次より前"
            kari_dic["text"]=text[:ind]
            text_dic["text"].append(kari_dic)
            kari_text=text[ind:]
  
    else:
        false_mokuzi.append(mokuzi)


In [301]:
text_dic

{'text': [{'title': '目次より前',
   'text': '※決算短信は監査の対象外です※業績予想の適切な利用に関する説明、その他特記事項本資料に記載されている業績見通し等の将来に関する記述は、当社が現在入手している情報及び合理的であると判断する一定の前提に基づいており、その達成を当社として約束する趣旨のものではありません。\nまた、実際の業績等は様々な要因により大きく異なる可能性があります。\n業績予想の前提となる条件及び業績予想のご利用にあたっての注意事項等は、添付資料４ページ「'},
  {'title': '１．経営成績等の概況', 'text': '（４）今後の見通し」をご覧ください。\n。\n１．経営成績等の概況'},
  {'title': '（１）当期の経営成績の概況',
   'text': '当連結会計年度におけるわが国経済は、政府の経済政策や金融政策を背景に緩やかな回復傾向が見られましたが、アジア新興国の景気減速やイギリスのＥＵ離脱問題、アメリカ新政権の政策の国内景気への影響等により、依然として先行き不透明な状況が続いております。\nこのような状況の中、主力であるいちご果実・青果事業におきましては、昨年９月の全国的な日照不足を端緒とする天候不順が特に上半期の業績に悪影響を及ぼしました。\nまた、馬鈴薯事業におきましては、昨年４月の熊本地震、昨年８月の北海道での台風被害による影響を大きく受けた年度となりました。\nこの結果、当連結会計年度の業績は、売上高は３，７２２，６３０千円（前期比１７．０％減少）、営業損失は１８３，２６９千円（前期は営業損失６８，７５２千円）、経常損失は１７７，０１３千円（前期は経常損失６０，４６６千円）となりました。\nまた、固定資産の減損損失による特別損失を計上したため、親会社株主に帰属する当期純損失は４０９，４９３千円（前期は親会社株主に帰属する当期純損失１１０，１３３千円）となりました。\n当連結会計年度の当社グループが営む事業は、いちご果実・青果事業、種苗事業、馬鈴薯事業、運送事業の４事業となっております。\nセグメントの業績は次のとおりであります。\n（いちご果実・青果事業）いちご果実・青果事業の主力商品は業務用いちご果実であります。\n夏秋期は自社品種いちごと輸入いちごを、その後は国産促成

In [247]:
kari_text.find(mokuzi_list[6])

3823

### 全てを回す

In [4]:
import os

['20200727467161.txt', '20220119569653.txt', '20190723474600.txt', '20211019413422.txt', '20220415522807.txt', '20170619410034.txt', '20210125447951.txt', '20191021410184.txt', '20200424499259.txt', '20171215436591.txt', '20180921409376.txt', '20200128451449.txt', '20180627471686.txt', '20190510420409.txt', '20210423498870.txt', '20201023407472.txt', '20180402403415.txt', '20210708463685.txt', '20170905468428.txt', '20190125463477.txt']


In [307]:
#全てを回す
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    df = pd.read_csv(df_path,index_col=0)
    dic =  '../data/text_data_ver2/text_data/'+str(company)
    
    #フォルダを作成する
    try:
        os.makedirs(dic)
    except FileExistsError:
        pass
    
    #企業ごとで回す
    for ind in df.index:
        pdf_path='../data/TDNet_full_data/API_data/'+str(company)+'_g/'+str(company)+'_'+str(df["pdf_code"][ind])+'.pdf'
        out = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(df["pdf_code"][ind])+'.txt'
    
      #textに変換
        file = pdf_path
        text=""
        sentences_list=[]
        #print(file)
        #print(out)
        with open(out,'w') as out:
            mat_eol = re.compile("\n".encode('utf-8'))
            #print(fp)
            sentences = []
            sentence=[]
            p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
            temp = u""
            #print(p.stdout.decode('utf-8'))
            for line in mat_eol.split(p.stdout):
                try:
                    line = line.decode('utf-8')
                except:
                    line = ""
                if line == "":
                    if temp != "" and u"。" in temp:
                        sentences = sentences + re.split(u"。", temp)
                    temp = u""
                else:
                    temp = temp+line

            if temp != "" and u"。" in temp:
                sentences = sentences + re.split(u"。", temp)

            for line in sentences :
                text=line

                #if len(text) <= 300 and len(text) >+ 3:
                #if len(text) > 3:
                sentences_list.append(normalize(text + u"。"))
                    #print(text)

            for sentence in sentences_list:
                #print(sentence)
                out.write(sentence+"\n")
    #break

100%|█████████████████████████████████████| 2512/2512 [1:02:29<00:00,  1.49s/it]


In [308]:
import requests
import sys

def main():
    url = "https://notify-api.line.me/api/notify"
    token = 'i8A6mltzBKu1Y2AJmiWz5l2SZpKacBRAbr7RWoCHJuI'
    headers = {"Authorization" : "Bearer "+ token}

    args = sys.argv
    if args[0] == 1:
        message =  '通常終了'
    else :
        message = '異常終了'
    payload = {"message" :  message}
    files = {"imageFile": open("end.jpg", "rb")}

    r = requests.post(url ,headers = headers ,params=payload, files=files)

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'end.jpg'

In [23]:
#全てを回す(indexを取得だとエラーがあるため)
for company in tqdm.tqdm(company_list.index):
    df_path = '../data/new2/df_achieve/this/{}.csv'.format(company)
    df = pd.read_csv(df_path,index_col=0)
    dic =  '../data/text_data_ver2/text_data/'+str(company)
   
    folderfile = os.listdir(dic)

    #企業ごとで回す
    for code in df["pdf_code"]:
        file_name = str(code)+'.txt'
        if file_name in folderfile:
            pass
        else:
            pdf_path='../data/TDNet_full_data/API_data/'+str(company)+'_g/'+str(company)+'_'+str(code)+'.pdf'
            out = '../data/text_data_ver2/text_data/'+str(company)+'/'+str(code)+'.txt'

          #textに変換
            file = pdf_path
            text=""
            sentences_list=[]
            #print(file)
            #print(out)
            with open(out,'w') as out:
                mat_eol = re.compile("\n".encode('utf-8'))
                #print(fp)
                sentences = []
                sentence=[]
                p = subprocess.run(('pdftotext', file, '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
                temp = u""
                #print(p.stdout.decode('utf-8'))
                for line in mat_eol.split(p.stdout):
                    try:
                        line = line.decode('utf-8')
                    except:
                        line = ""
                    if line == "":
                        if temp != "" and u"。" in temp:
                            sentences = sentences + re.split(u"。", temp)
                        temp = u""
                    else:
                        temp = temp+line

                if temp != "" and u"。" in temp:
                    sentences = sentences + re.split(u"。", temp)

                for line in sentences :
                    text=line

                    #if len(text) <= 300 and len(text) >+ 3:
                    #if len(text) > 3:
                    sentences_list.append(normalize(text + u"。"))
                        #print(text)

                for sentence in sentences_list:
                    #print(sentence)
                    out.write(sentence+"\n")

100%|███████████████████████████████████████| 2512/2512 [00:28<00:00, 87.76it/s]


## memo

In [4]:
for i in range(0,1):
    with open("out.txt",'w') as out:
        mat_eol = re.compile("\n".encode('utf-8'))
        #print(fp)
        sentences = []
        sentence=[]
        p = subprocess.run(('pdftotext', "rakuten.pdf", '-'), stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        temp = u""
        for line in mat_eol.split(p.stdout):
            try:
                line = line.decode('utf-8')
            except:
                line = ""
            if line == "":
                if temp != "" and u"。" in temp:
                    sentences = sentences + re.split(u"。", temp)
                temp = u""
            else:
                temp = temp+line

        if temp != "" and u"。" in temp:
            sentences = sentences + re.split(u"。", temp)

        sentences = [normalize(line + u"。")
                    for line in sentences if len(line) <= 300 and len(line) >+ 3]
        for sentence in sentences:
            out.write(sentence+"\n")